In [81]:
import pandas as pd
import numpy as np
import difflib

In [94]:
## Construct category list
#program_f = pd.read_csv('handson_data/handson_program.csv')
cats = ['Assistance and Support for Elderly',
        'Disaster & Emergency Services',
        'Education and Empowerment for Children and Youth',
        'Empowerment and Support for Domestic and Migrant Workers', 
        'Environmental Conservation',
        'Education and Empowerment for Refugees and Asylum Seekers',
        'Health and Wellness',
        'Hunger & Homelessness', 
        'Support for Homeless People',
        'Adult Education',
        'International Service',
        'Internships & Employment',
        'Justice & Legal Services',
        'Refugee & Asylum Seekers Services',
        'Schools',
        'Sports & Recreation',
        'Technology',
        'Family Services',
        'Arts & Culture',
        'Civic & Community']
print(len(cats))
cats_abbr = ['interest_elderly','interest_disaster','interest_education_young',
             'interest_worker','interest_environment','interest_education_refugee',
             'interest_health','interest_hunger','interest_homeless',
             'interest_education_adult','interest_international','interest_internship',
             'interest_justice','interest_refugee','interest_school','interest_sports',
             'interest_tech','interest_family','interest_arts','interest_civic']
cats_dict = {cats_abbr[i]:cats[i] for i in range(len(cats))}

20


In [95]:
cats_dict

{'interest_elderly': 'Assistance and Support for Elderly',
 'interest_disaster': 'Disaster & Emergency Services',
 'interest_education_young': 'Education and Empowerment for Children and Youth',
 'interest_worker': 'Empowerment and Support for Domestic and Migrant Workers',
 'interest_environment': 'Environmental Conservation',
 'interest_education_refugee': 'Education and Empowerment for Refugees and Asylum Seekers',
 'interest_health': 'Health and Wellness',
 'interest_hunger': 'Hunger & Homelessness',
 'interest_homeless': 'Support for Homeless People',
 'interest_education_adult': 'Adult Education',
 'interest_international': 'International Service',
 'interest_internship': 'Internships & Employment',
 'interest_justice': 'Justice & Legal Services',
 'interest_refugee': 'Refugee & Asylum Seekers Services',
 'interest_school': 'Schools',
 'interest_sports': 'Sports & Recreation',
 'interest_tech': 'Technology',
 'interest_family': 'Family Services',
 'interest_arts': 'Arts & Culture

In [40]:
## construct skill set
skills = ['Accounting / finance',
          'Administrative/general office support',
          'Communications and marketing',
          'Fundraising',
          'Gardening',
          'Graphic design',
          'Health / medical',
          'Event management',
          'Human resources',
          'IT',
          'Legal',
          'Photography',
          'Project management',
          'Teaching/Training/Fine arts',
          'Teaching/Training/Fitness/sports',
          'Teaching/Training/Music/drama',
          'Teaching/Training/Health',
          'Teaching/Training/IT',
          'Translation',
          'Videography']
print(len(skills))

20


In [41]:
## Construct feature map for user
feature = ['gender_female','gender_male',
           'employment_pt','employment_ft','employment_retired','employment_others',
           'hours_served_total','hours_served_oppo','oppo_cat',
           'interest_elderly','interest_disaster','interest_education_young',
           'interest_worker','interest_environment','interest_education_refugee',
           'interest_health','interest_hunger','interest_homeless',
           'interest_education_adult','interest_international','interest_internship',
           'interest_justice','interest_refugee','interest_school','interest_sports',
           'interest_tech','interest_family','interest_arts','interest_civic']

In [42]:
## Import user & program data
user_f = pd.read_csv('handson_data/dummy_volunteer_profile.csv')
program_f = pd.read_csv('handson_data/handson_program.csv')

In [54]:
## Map between event name and cat
program_cat_match = program_f[['impact_area','opportunity_name']]
program_cat_match = program_cat_match.dropna(how='all')
program_cat_match = program_cat_match.drop_duplicates()
program_cat_match

,impact_area,opportunity_name
0,Environmental Conservation,Serve-A-Thon: Soap Recycling 1:30 p.m. session...
2,Environmental Conservation,Serve-A-Thon: Soap Recycling 10 a.m. session (...
4,Hunger & Homelessness,Serve-A-Thon: Deliver Daily Necessities to Hom...
6,Environmental Conservation,Serve-A-Thon: Help with sorting donated goods ...
13,Hunger & Homelessness,Youth Opportunity: Collect Bread for Redistrib...
...,...,...
462,Hunger & Homelessness,Youth Empowered Opportunity: Collect Bread for...
468,Education and Empowerment for Children and Youth,Youth Empowered Info Session: Providing suppor...
469,Environmental Conservation,Youth Empowered Opportunity: Info Session on R...
470,Education and Empowerment for Children and Youth,Youth Empowered Info Session: Be a 'Blind Budd...


In [61]:
## Turn map to dict
program_cat_match_dict = program_cat_match.set_index('opportunity_name').T.to_dict('list')

<ipython-input-61-8fef1c2c1726>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  program_cat_match_dict = program_cat_match.set_index('opportunity_name').T.to_dict('list')


In [72]:
## Construct feature matrix from user data
user_matrix_id = user_f.connection_id
user_matrix_gender_female = [user_f.gender == 'Female']
user_matrix_gender_male = [user_f.gender == 'Male']
user_matrix_employment_pt = [user_f.employment_status == 'Part Time']
user_matrix_employment_ft = [user_f.employment_status == 'Full Time']
user_matrix_employment_retired = [user_f.employment_status == 'Retired']
d = {'Other': True, 'Not Employed': True, 'Full Time': False, 'Part Time': False}
user_matrix_employment_others = user_f.employment_status.map(d)
user_matrix_employment_others = user_matrix_employment_others.fillna(True)
user_matrix_hours_served_total = user_f.hours_served_all
user_matrix_hours_served_oppo = user_f.hours_served_opportunity
user_matrix_oppo_cat = user_f.opportunity_name.map(program_cat_match_dict)

In [99]:
## Construct feature matrix from user interest
for cat_abbr in cats_abbr:
    tmp_pdseries = user_f.interest_list.str.contains(cats_dict[cat_abbr])
    tmp_pdseries = tmp_pdseries.fillna(False)
    vars()['user_matrix_'+cat_abbr] = tmp_pdseries

0     False
1     False
2     False
3     False
4     False
      ...  
94     True
95     True
96     True
97    False
98    False
Name: interest_list, Length: 99, dtype: bool


In [ ]:
## Build ones for 

In [86]:
user_matrix_oppo_cat.loc[98][0] in 'Hunger & Homelessness' or 'Hunger & Homelessness' in user_matrix_oppo_cat.loc[98][0]

True